# Amplifier Mount Plans: From Profiles to Kernel Configuration

This notebook demonstrates how amplifier-core (the library) converts profiles and settings into mount plans that the Amplifier kernel understands.

## Important Context

**This notebook demonstrates amplifier-core library concepts.** The amplifierd REST API implements these concepts but adds:
- Profile caching and compilation (refs resolved to local files)
- REST API layer for remote access
- Storage in `$AMPLIFIERD_HOME/share/profiles/{collection}/{profile}/`
- Cached agent and context files

**Key difference**: In amplifierd, profiles are compiled (all git+https refs resolved and cached) BEFORE mount plans are generated. The mount plan then references local cached files.

## Overview

The conversion flow follows these steps:

```
User Intent (CLI + flags)
    ↓
Profile Loading (YAML + references)
    ↓
Settings Merge (local/project/global)
    ↓
Compilation → Mount Plan
    ↓
Kernel Validates & Loads
    ↓
Session Ready
```

## Setup: Import Required Libraries and Define Helper Functions

In [ ]:
import json
import os
import re
from pprint import pprint

# Helper functions for demonstration (in real amplifier-core, these are in the library)

def deep_merge(base: dict, overlay: dict) -> dict:
    """Merge two configurations deeply.
    
    - Module lists (providers, tools, hooks): Merged by module ID
    - Config objects: Recursively merged
    - Primitives: Child value overrides parent
    """
    result = base.copy()
    for key, value in overlay.items():
        if key in result:
            if isinstance(result[key], dict) and isinstance(value, dict):
                # Recursively merge dictionaries
                result[key] = deep_merge(result[key], value)
            elif isinstance(result[key], list) and isinstance(value, list):
                # For module lists, merge by module ID
                if key in ['providers', 'tools', 'hooks', 'agents']:
                    result[key] = merge_module_lists(result[key], value)
                else:
                    # For other lists, replace
                    result[key] = value
            else:
                # Overlay value replaces base value
                result[key] = value
        else:
            result[key] = value
    return result

def merge_module_lists(base: list, overlay: list) -> list:
    """Merge two module lists, matching by 'module' field."""
    result = base.copy()
    for overlay_item in overlay:
        if not isinstance(overlay_item, dict) or 'module' not in overlay_item:
            result.append(overlay_item)
            continue
            
        # Find matching module in base
        module_id = overlay_item['module']
        matched = False
        for i, base_item in enumerate(result):
            if isinstance(base_item, dict) and base_item.get('module') == module_id:
                # Merge this specific module config
                result[i] = deep_merge(base_item, overlay_item)
                matched = True
                break
        
        if not matched:
            # New module, append it
            result.append(overlay_item)
    
    return result

def expand_env_vars(config: dict) -> dict:
    """Expand ${VAR} and ${VAR:default} in config."""
    def expand_value(value):
        if isinstance(value, str):
            # Pattern: ${VAR} or ${VAR:default}
            pattern = r'\$\{([^:}]+)(?::([^}]+))?\}'
            def replacer(match):
                var_name, default = match.groups()
                return os.environ.get(var_name, default or '')
            return re.sub(pattern, replacer, value)
        elif isinstance(value, dict):
            return {k: expand_value(v) for k, v in value.items()}
        elif isinstance(value, list):
            return [expand_value(v) for v in value]
        return value
    
    return expand_value(config)

print("✓ Helper functions loaded")

## Part 1: Understanding Mount Plan Structure

A mount plan is the contract between the application layer and the kernel. It's a plain Python dictionary with these key sections:

- **session**: Core session configuration (orchestrator, context, limits)
- **providers**: LLM backends (Anthropic, OpenAI, etc.)
- **tools**: Agent capabilities (filesystem, bash, web, etc.)
- **hooks**: Observability and lifecycle hooks
- **agents**: Named configuration overlays for delegation

In [ ]:
# Minimal valid mount plan
minimal_mount_plan = {
    "session": {"orchestrator": "loop-basic", "context": "context-simple"},
    "providers": [
        {
            "module": "provider-mock",
            "source": "git+https://github.com/microsoft/amplifier-module-provider-mock@main"
        }
    ],
}

print("Minimal Mount Plan:")
pprint(minimal_mount_plan)

## Part 2: A More Complete Mount Plan

Let's build a more realistic development mount plan with multiple modules and configuration.

**Note**: Every module reference MUST include a `source:` field specifying where to fetch it from.

In [ ]:
# Development mount plan with full configuration
dev_mount_plan = {
    "session": {
        "orchestrator": "loop-streaming",
        "orchestrator_source": "git+https://github.com/microsoft/amplifier-module-loop-streaming@main",
        "context": "context-persistent",
        "context_source": "git+https://github.com/microsoft/amplifier-module-context-persistent@main",
        "injection_budget_per_turn": 10000,
        "injection_size_limit": 10240,
    },
    "context": {"config": {"max_tokens": 200000, "compact_threshold": 0.92, "auto_compact": True}},
    "providers": [
        {
            "module": "provider-anthropic",
            "source": "git+https://github.com/microsoft/amplifier-module-provider-anthropic@main",
            "config": {"model": "claude-sonnet-4-5", "api_key": "${ANTHROPIC_API_KEY}", "debug": True},
        }
    ],
    "tools": [
        {
            "module": "tool-filesystem",
            "source": "git+https://github.com/microsoft/amplifier-module-tool-filesystem@main",
            "config": {"allowed_paths": ["."], "require_approval": False},
        },
        {
            "module": "tool-bash",
            "source": "git+https://github.com/microsoft/amplifier-module-tool-bash@main"
        },
    ],
    "hooks": [
        {
            "module": "hooks-logging",
            "source": "git+https://github.com/microsoft/amplifier-module-hooks-logging@main",
            "config": {"output_dir": ".amplifier/logs", "mode": "session-only"},
        }
    ],
}

print("Development Mount Plan:")
print(json.dumps(dev_mount_plan, indent=2))

## Part 3: Deep Merging - How Profiles Combine

When profiles reference other profiles (via URIs), or when settings override profile values, Amplifier uses **deep merging** to combine configurations intelligently.

### Deep Merge Strategy

- **Module lists** (providers, tools, hooks): Merged by module ID
- **Config objects**: Recursively merged
- **Primitives**: Child value overrides parent

In [ ]:
# Parent profile configuration
parent_config = {
    "providers": [
        {
            "module": "provider-anthropic",
            "source": "git+https://github.com/microsoft/amplifier-module-provider-anthropic@main",
            "config": {"model": "claude-sonnet-4-5", "temperature": 0.7, "max_tokens": 4096},
        }
    ]
}

# Child profile configuration (references parent, then overlays)
child_config = {
    "providers": [
        {
            "module": "provider-anthropic",
            "config": {
                "debug": True,
                "temperature": 0.9,  # Override parent's temperature
            },
        }
    ]
}

print("Parent Config:")
pprint(parent_config)
print("\nChild Config:")
pprint(child_config)

# Merge them using deep_merge
merged_config = deep_merge(parent_config, child_config)

print("\nMerged Config:")
pprint(merged_config)
print("\nNotice how:")
print("  ✓ 'source' was inherited from parent")
print("  ✓ 'model' and 'max_tokens' were inherited from parent")
print("  ✓ 'temperature' was overridden by child (0.9)")
print("  ✓ 'debug' was added by child")

## Part 4: Profile Schema v2 - Reference-Based Composition

Modern amplifier profiles (schema v2) use **reference URIs** instead of inheritance. This is more flexible and explicit.

### URI Formats Supported

- `git+https://github.com/org/repo@ref#subdirectory=path` - Git repository
- `https://raw.githubusercontent.com/org/repo/ref/file.md` - Direct HTTPS
- `file:///absolute/path/to/file.md` - Local filesystem
- Relative paths for project-local files

In [ ]:
# Example profile using schema v2 reference-based composition
schema_v2_profile = {
    "version": "2.0",
    "name": "research-assistant",
    
    # Reference agents from various sources
    "agents": [
        "git+https://github.com/amplifier-community/agents@main#subdirectory=research/web-researcher.md",
        "https://raw.githubusercontent.com/myorg/agents/main/analysts/data-analyst.md",
        "./agents/custom-researcher.md",  # Local to profile directory
    ],
    
    # Reference context files
    "context": [
        "git+https://github.com/amplifier-community/context@main#subdirectory=research-guidelines/",
        "./context/project-specific.md",
    ],
    
    # Module configurations with sources
    "providers": [
        {
            "module": "provider-anthropic",
            "source": "git+https://github.com/microsoft/amplifier-module-provider-anthropic@main",
            "config": {"model": "claude-sonnet-4-5"}
        }
    ],
    
    "tools": [
        {
            "module": "tool-web",
            "source": "git+https://github.com/microsoft/amplifier-module-tool-web@main",
            "config": {"timeout": 30}
        },
        {
            "module": "tool-search",
            "source": "git+https://github.com/microsoft/amplifier-module-tool-search@main"
        }
    ]
}

print("Schema v2 Profile (Reference-Based):")
print(json.dumps(schema_v2_profile, indent=2))

print("\n" + "="*60)
print("Key Points:")
print("="*60)
print("✓ No 'extends' - explicit URI references instead")
print("✓ Agents and context fetched from git/https/file sources")
print("✓ Each module has explicit 'source' field")
print("✓ amplifierd caches all refs before generating mount plan")

## Part 5: Environment Variable Expansion

Mount plans support environment variable references using `${VAR_NAME}` syntax, with optional defaults using `${VAR_NAME:default}`.

In [ ]:
# Set some example environment variables
os.environ["ANTHROPIC_API_KEY"] = "sk-ant-example-key-123"
os.environ["LOG_LEVEL"] = "debug"

# Mount plan with environment variable references
config_with_envvars = {
    "providers": [
        {
            "module": "provider-anthropic",
            "source": "git+https://github.com/microsoft/amplifier-module-provider-anthropic@main",
            "config": {
                "api_key": "${ANTHROPIC_API_KEY}",  # Will be replaced
                "debug": "${DEBUG_MODE:false}",  # Will use default "false"
                "log_level": "${LOG_LEVEL}",  # Will be replaced with "debug"
            },
        }
    ]
}

print("Before expansion:")
pprint(config_with_envvars)

# Expand environment variables
expanded_config = expand_env_vars(config_with_envvars)

print("\nAfter expansion:")
pprint(expanded_config)

## Part 6: Adding Multiple Modules

Let's demonstrate how base configuration and overlays combine when adding multiple tools and providers.

In [ ]:
# Base configuration with basic tools
base_config = {
    "tools": [
        {
            "module": "tool-filesystem",
            "source": "git+https://github.com/microsoft/amplifier-module-tool-filesystem@main",
        },
        {
            "module": "tool-bash",
            "source": "git+https://github.com/microsoft/amplifier-module-tool-bash@main"
        },
    ]
}

# Overlay adding web tool and modifying filesystem config
overlay_config = {
    "tools": [
        {"module": "tool-filesystem", "config": {"allowed_paths": ["/home/user/projects"], "require_approval": True}},
        {
            "module": "tool-web",
            "source": "git+https://github.com/microsoft/amplifier-module-tool-web@main",
            "config": {"timeout": 30},
        },
    ]
}

print("Base Configuration:")
pprint(base_config)
print("\nOverlay Configuration:")
pprint(overlay_config)

# Merge them
final_config = deep_merge(base_config, overlay_config)

print("\nFinal Merged Configuration:")
pprint(final_config)
print("\nNotice how:")
print("  ✓ tool-filesystem kept its source from base")
print("  ✓ tool-filesystem gained config from overlay")
print("  ✓ tool-bash remained unchanged")
print("  ✓ tool-web was added from overlay")

## Part 7: Agent Delegation Configuration

The `agents` section is special - it contains named configuration overlays for spawning specialized child sessions, not modules to load immediately.

When the main agent wants to delegate work, it uses these agent configurations to spawn a child session with customized capabilities.

In [ ]:
# Mount plan with agent delegation support
delegation_mount_plan = {
    "session": {"orchestrator": "loop-streaming", "context": "context-simple"},
    "providers": [
        {
            "module": "provider-anthropic",
            "source": "git+https://github.com/microsoft/amplifier-module-provider-anthropic@main",
            "config": {"model": "claude-sonnet-4-5"},
        }
    ],
    "tools": [
        {
            "module": "tool-filesystem",
            "source": "git+https://github.com/microsoft/amplifier-module-tool-filesystem@main"
        },
        {
            "module": "tool-bash",
            "source": "git+https://github.com/microsoft/amplifier-module-tool-bash@main"
        },
        {
            "module": "tool-task",
            "source": "git+https://github.com/microsoft/amplifier-module-tool-task@main"
        },  # This enables delegation!
    ],
    "agents": {
        # Specialized debugging agent with different model
        "bug-hunter": {
            "description": "Specialized debugging agent",
            "providers": [
                {
                    "module": "provider-anthropic",
                    "source": "git+https://github.com/microsoft/amplifier-module-provider-anthropic@main",
                    "config": {
                        "model": "claude-opus-4-1",  # Upgrade to Opus
                        "temperature": 0.3,  # Lower for precision
                    },
                }
            ],
            "tools": [
                {
                    "module": "tool-filesystem",
                    "source": "git+https://github.com/microsoft/amplifier-module-tool-filesystem@main"
                },
                {
                    "module": "tool-bash",
                    "source": "git+https://github.com/microsoft/amplifier-module-tool-bash@main"
                },
                # Note: no tool-task (prevent recursive delegation)
            ],
            "system": {"instruction": "You are a debugging specialist. Focus on finding root causes."},
        },
        # Fast research agent with limited tools
        "researcher": {
            "description": "Fast research and information gathering",
            "providers": [
                {
                    "module": "provider-anthropic",
                    "source": "git+https://github.com/microsoft/amplifier-module-provider-anthropic@main",
                    "config": {
                        "model": "claude-haiku-4-1",  # Downgrade for speed
                        "temperature": 0.5,
                    },
                }
            ],
            "tools": [
                {
                    "module": "tool-web",
                    "source": "git+https://github.com/microsoft/amplifier-module-tool-web@main"
                },
                {
                    "module": "tool-filesystem",
                    "source": "git+https://github.com/microsoft/amplifier-module-tool-filesystem@main"
                },  # Read-only effectively
            ],
            "system": {"instruction": "You are a research specialist. Gather information quickly and concisely."},
        },
    },
}

print("Mount Plan with Agent Delegation:")
print(json.dumps(delegation_mount_plan, indent=2))

print("\n" + "="*60)
print("Key Points:")
print("="*60)
print("✓ Main session has tool-task enabled for delegation")
print("✓ 'bug-hunter' agent uses Opus for complex debugging")
print("✓ 'researcher' agent uses Haiku for fast information gathering")
print("✓ Each agent has custom system instructions")
print("✓ Agents can't recursively delegate (no tool-task)")
print("✓ All modules have explicit 'source' fields")

## Part 8: The Complete Flow - CLI to Mount Plan

Let's simulate the complete flow from CLI invocation to mount plan, as it happens in amplifier-core.

This demonstrates the full configuration resolution:

1. Start with defaults
2. Apply profile configuration (refs already resolved by amplifierd)
3. Merge settings overlays
4. Apply CLI overrides
5. Expand environment variables

In [ ]:
# Step 1: Default configuration
default_config = {
    "session": {
        "orchestrator": "loop-basic",
        "context": "context-simple",
    },
    "providers": [],
    "tools": [],
    "hooks": [],
}

print("Step 1 - Default Config:")
pprint(default_config)

# Step 2: Profile configuration (simulating compiled/cached profile)
# In amplifierd, all git+https refs are already resolved to local cached files
profile_config = {
    "session": {
        "orchestrator": "loop-streaming",
        "orchestrator_source": "git+https://github.com/microsoft/amplifier-module-loop-streaming@main",
        "context": "context-persistent",
        "context_source": "git+https://github.com/microsoft/amplifier-module-context-persistent@main",
    },
    "providers": [
        {
            "module": "provider-anthropic",
            "source": "git+https://github.com/microsoft/amplifier-module-provider-anthropic@main",
            "config": {"model": "claude-sonnet-4-5"},
        }
    ],
    "tools": [
        {
            "module": "tool-filesystem",
            "source": "git+https://github.com/microsoft/amplifier-module-tool-filesystem@main"
        },
        {
            "module": "tool-bash",
            "source": "git+https://github.com/microsoft/amplifier-module-tool-bash@main"
        }
    ],
}

config = deep_merge(default_config, profile_config)
print("\nStep 2 - After Profile Merge:")
pprint(config)

# Step 3: Settings overlay (from user settings)
settings_overlay = {
    "tools": [
        {
            "module": "tool-web",
            "source": "git+https://github.com/microsoft/amplifier-module-tool-web@main"
        }
    ],
    "hooks": [
        {
            "module": "hooks-logging",
            "source": "git+https://github.com/microsoft/amplifier-module-hooks-logging@main",
            "config": {"mode": "all"}
        }
    ],
}

config = deep_merge(config, settings_overlay)
print("\nStep 3 - After Settings Merge:")
pprint(config)

# Step 4: CLI overrides (from command line flags like --model)
cli_overrides = {
    "providers": [
        {
            "module": "provider-anthropic",
            "config": {
                "model": "claude-opus-4-1",  # Override model via --model flag
                "api_key": "${ANTHROPIC_API_KEY}",
            },
        }
    ]
}

config = deep_merge(config, cli_overrides)
print("\nStep 4 - After CLI Overrides:")
pprint(config)

# Step 5: Expand environment variables
os.environ["ANTHROPIC_API_KEY"] = "sk-ant-api-key-from-env"
final_mount_plan = expand_env_vars(config)

print("\nStep 5 - Final Mount Plan (after env expansion):")
print(json.dumps(final_mount_plan, indent=2))

print("\n" + "="*60)
print("Compilation Summary:")
print("="*60)
print("✓ Orchestrator upgraded from 'loop-basic' to 'loop-streaming'")
print("✓ Context upgraded from 'context-simple' to 'context-persistent'")
print("✓ Provider added from profile")
print("✓ Model overridden via CLI: 'claude-opus-4-1'")
print("✓ Tools combined: filesystem, bash, web")
print("✓ Hooks added from settings")
print("✓ API key resolved from environment")
print("✓ All modules have explicit source URIs")

## Part 9: Production Configuration Best Practices

Let's create a production-ready mount plan with safety controls and cost management.

In [ ]:
production_mount_plan = {
    "session": {
        "orchestrator": "loop-events",
        "orchestrator_source": "git+https://github.com/microsoft/amplifier-module-loop-events@main",
        "context": "context-persistent",
        "context_source": "git+https://github.com/microsoft/amplifier-module-context-persistent@main",
        # Conservative injection limits for cost control
        "injection_budget_per_turn": 500,
        "injection_size_limit": 8192,
    },
    "context": {
        "config": {
            "max_tokens": 200000,
            "compact_threshold": 0.95,  # More aggressive compaction
            "auto_compact": True,
        }
    },
    "providers": [
        {
            "module": "provider-anthropic",
            "source": "git+https://github.com/microsoft/amplifier-module-provider-anthropic@main",
            "config": {
                "model": "claude-sonnet-4-5",
                "api_key": "${ANTHROPIC_API_KEY}",
                "max_tokens": 4096,  # Cap output length
                "debug": False,  # No debug logs in production
            },
        }
    ],
    "tools": [
        {
            "module": "tool-filesystem",
            "source": "git+https://github.com/microsoft/amplifier-module-tool-filesystem@main",
            "config": {
                "allowed_paths": ["/app/data"],  # Restricted paths
                "require_approval": True,  # Human approval required
            },
        }
    ],
    "hooks": [
        {
            "module": "hooks-logging",
            "source": "git+https://github.com/microsoft/amplifier-module-hooks-logging@main",
            "config": {
                "mode": "all",
                "redaction_enabled": True,  # Redact sensitive data
            },
        },
        {
            "module": "hooks-backup",
            "source": "git+https://github.com/microsoft/amplifier-module-hooks-backup@main"
        },
    ],
}

print("Production Mount Plan with Safety Controls:")
print(json.dumps(production_mount_plan, indent=2))

print("\n" + "="*60)
print("Production Safety Features:")
print("="*60)
print("✓ Conservative injection limits (500 tokens/turn, 8KB max)")
print("✓ Aggressive context compaction (0.95 threshold)")
print("✓ Output token cap (4096 max)")
print("✓ Restricted filesystem access (/app/data only)")
print("✓ Human approval required for file operations")
print("✓ Comprehensive logging with redaction")
print("✓ Automatic session backup")
print("✓ Debug mode disabled")
print("✓ All modules have explicit source URIs")

## Part 10: amplifierd Storage Structure

When amplifierd compiles profiles, it caches all referenced resources. Here's where things live:

```
$AMPLIFIERD_HOME/share/profiles/
├── {collection}/
│   └── {profile}/
│       ├── profile.yaml           # Compiled profile
│       ├── agents/               # Cached agent files
│       │   ├── researcher.md
│       │   └── coder.md
│       ├── context/              # Cached context files
│       │   ├── guidelines.md
│       │   └── examples.md
│       └── modules/              # Cached module code
│           ├── provider-anthropic/
│           └── tool-filesystem/
```

**Key points**:
- Profiles are compiled: all git+https refs resolved to local files
- Mount plans reference these cached local paths
- REST API serves from this cache
- `amplifierd sync` updates the cache

## Summary

This notebook demonstrated:

1. **Mount Plan Structure**: The kernel's contract format
2. **Deep Merging**: How profiles and settings intelligently combine
3. **Schema v2 References**: URI-based composition (git+https, https, file)
4. **Environment Variables**: Dynamic configuration resolution
5. **Module Management**: Adding, overriding, and configuring modules (with sources!)
6. **Agent Delegation**: Specialized child session configurations
7. **Complete Flow**: From CLI to final mount plan
8. **Production Best Practices**: Safety and cost controls
9. **amplifierd Storage**: How profiles are cached and compiled

### Key Takeaways

- **Mount plans** are the single format the kernel understands
- **Profiles** compile to mount plans through reference resolution and merging
- **Deep merging** preserves sources and combines configs intelligently
- **Every module MUST have a source URI**
- **Environment variables** enable dynamic, secure configuration
- **Agent sections** are overlays for delegation, not immediate loads
- **Production configs** should include safety limits and approval gates
- **amplifierd compiles profiles** before generating mount plans (refs → cache → mount plan)

### Related Documentation

- Profiles Guide
- Modules Guide
- CLI Guide
- Mount Plans Guide